In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import os
import pandas as pd
from sklearn.metrics import (
    average_precision_score,
    precision_recall_curve,
    roc_auc_score,
)
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import urllib.request
import multiprocessing as mp

from datetime import date
today = date.today()
today = today.strftime("%Y-%m-%d")

<IPython.core.display.Javascript object>

In [3]:
from ananse.benchmark import (
    read_reference,
    download_trrust_reference,
    download_msigdb_reference,
    validate_files,
    read_networks,
)

<IPython.core.display.Javascript object>

# Overview

This notebook contains the basic code to benchmark GRNs using a variety of references. The data is not included (due to size, licensing, etc.), but instructions are provided to download them.

**Please note:** the analysis is currently not optimized and uses **a lot** of memory.


# Obtaining GRN reference databases

We do not include the reference databases with this repository, as the license information is unclear and/or the files are big. Some can be downloaded automatically, however, some need to be downloaded by hand. 

## Download instructions

* RegNetwork:

    * Go to the following page: http://regnetworkweb.org/search.jsp, set "Confidence" to "High" and press "Search"
    * Set "Export as" to "CSV", press "Export" and save the resulting `.csv` file.

* DoRothEA:

    * Download the `entire_database.rda` file from https://github.com/saezlab/dorothea/tree/master/data
    * Convert the file using R to a tab-seperated text file.




In [4]:
# Downloading references that can be automatically downloaded

# TRRUST
if not os.path.exists("../data/references/trrust.txt"):
    download_trrust_reference("../data/references/trrust.txt")

# GO and Expression Correlation benchmark (see ANANSE manuscript)
for fname in ["go_net.txt.gz", "expressioncorrelation.txt.gz"]:
    if not os.path.exists(f"../data/references/{fname}"):
        print(f"Downloading {fname}")
        urllib.request.urlretrieve(
            f"https://zenodo.org/record/4811884/files/{fname}",
            f"../data/references/{fname}",
        )

# TF perturbation data from Enrichr
if not os.path.exists("../data/references/Enrichr_TF_perturbations.txt"):
    print("Downloading Enrichr TF perturbation reference")
    urllib.request.urlretrieve(
        "https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=TF_Perturbations_Followed_by_Expression",
        "../data/references/Enrichr_TF_perturbations.txt",
    )

# MSigDB c3
msig_name = "../data/references/MSigDB_c3_tft_v72.txt"
if not os.path.exists(msig_name):
    print("Downloading MSigDB reference")
    download_msigdb_reference(msig_name)

Download MSigDB reference


<IPython.core.display.Javascript object>

# Obtaining GRNs to be evaluated

* ANANSE networks can be downloaded from https://doi.org/10.5281/zenodo.4807801
* GRNBoost2 networks were inferred on GTEx (see the notebook gtex_grn_with_arboreto.ipynb) and can be downloaded from https://doi.org/10.5281/zenodo.4814016
* GRNdb networks (inferred using SCENIC on single cell data) can be downloaded from http://www.grndb.com/
* PANDA networks (inferred on GTEx) can be downloaded from https://sites.google.com/a/channing.harvard.edu/kimberlyglass/tools/gtex-networks
* ARACNE and corto  networks (inferred on GTEx) were originally downloaded from https://www.dropbox.com/sh/qzl8vjeoa7mqxfp/AACEfLQpAzUz7rqqEHEjMrhQa?dl=0, however, these are now deleted :(

Define networks to use below, with `{}` where the tissue name would be:

In [21]:
IN_NETWORKS = {
    #     "GRNBoost2": "../data/grns/GRNBoost2/gtex.{}.grnboost2.feather",
    #     "ANANSE": "../data/grns/ANANSE/{}.network.txt.gz",
    "dummy": "../data/grns/dummy/{}.txt",
}

<IPython.core.display.Javascript object>

In [5]:
# Read references
dorothea = read_reference("dorothea", "../data/references/entire_database.txt")
perturb = read_reference(
    "perturbation", "../data/references/Enrichr_TF_perturbations.txt"
)
trrust = read_reference("trrust", "../data/references/trrust.txt")
corr = read_reference("correlation", f"../data/references/expressioncorrelation.txt.gz")
gonet = read_reference("goterm", f"../data/references//go_net.txt.gz")
msi = read_reference("msigdb", msig_name)
regnet = read_reference("regnet", "../data/references/regnet_highconf.csv")

2021-05-26 18:57:57 | INFO | Dorothea reference - 1243 TFs, 26459 targets, 1019256 edges.
2021-05-26 18:58:55 | INFO | TF perturbation reference - 330 TFs, 18097 targets, 231757 edges.
2021-05-26 18:59:13 | INFO | Using 'perturb_interaction' as interaction column.
2021-05-26 18:59:22 | INFO | Using 'interaction' as interaction column.
2021-05-26 19:01:28 | INFO | Using 'iscorrelation' as interaction column.
2021-05-26 19:04:11 | INFO | Using 'isgo' as interaction column.
2021-05-26 19:04:44 | INFO | Using 'interaction' as interaction column.
2021-05-26 19:05:38 | INFO | Using 'interaction' as interaction column.


<IPython.core.display.Javascript object>

In [40]:
# Select references for benchmark
references = [
    [dorothea, "dorothea", "is_evidence_curated"],
    [perturb, "TF perturbations", "interaction"],
    [trrust, "TRRUST", "interaction"],
    [corr, "Correlation", "interaction"],
    [gonet, "GO Term", "interaction"],
    [msi, "MSigDB", "interaction"],
    [regnet, "regNetwork", "interaction"],
]

<IPython.core.display.Javascript object>

In [36]:
# Select tissues
tissues = ["heart"]

<IPython.core.display.Javascript object>

In [37]:
def run_tissue(tissue):
    benchmark = []
    in_networks = {k: v.format(tissue) for k, v in IN_NETWORKS.items()}

    validate_files(in_networks.values(), ignore_missing=True)

    c = read_networks(in_networks)
    for network, name, ref_col in references:
        cols = c.columns
        test = network.join(c)
        test[network.columns.tolist()] = test[network.columns.tolist()].fillna(0)
        test = test.fillna(-100)

        print(f"{name}\t{tissue}\tbaseline\t{test[ref_col].mean():0.5f}")
        benchmark.append([name, tissue, "baseline", test[ref_col].mean(), 0.5])
        for col in cols:
            pr_auc = average_precision_score(test[ref_col], test[col])
            roc_auc = roc_auc_score(test[ref_col], test[col])
            print(f"{name}\t{tissue}\t{col}\t{pr_auc:0.5f}")
            benchmark.append([name, tissue, col, pr_auc, roc_auc])
    return benchmark

<IPython.core.display.Javascript object>

In [38]:
OUTFILE = f"{today}.benchmark_grns.txt"

<IPython.core.display.Javascript object>

In [39]:
pool = mp.Pool(2)  # increase for lower run-time but higher memory use
jobs = []

for tissue in tissues:
    jobs.append(pool.apply_async(run_tissue, (tissue,)))

resultdic = {}
for tissue, j in zip(tissues, jobs):
    resultdic[tissue] = j.get()

with open(OUTFILE, "w") as fl:
    fl.write("reference\ttissue\tnetwork\tpr_auc\troc_auc\n")
    for tissue, j in zip(tissues, jobs):
        resultdic[tissue] = j.get()
        for jj in resultdic[tissue]:
            for k in jj:
                fl.write(f"{k}\t")
            fl.write("\n")

pool.close()

2021-05-26 20:11:23 | INFO | Reading dummy
2021-05-26 20:11:23 | INFO | Merging dummy


TF perturbations	heart	baseline	0.03259
TF perturbations	heart	dummy	0.03544
regNetwork	heart	baseline	0.01042
regNetwork	heart	dummy	0.01139


<IPython.core.display.Javascript object>

In [41]:
!cat {OUTFILE}

reference	tissue	network	pr_auc	roc_auc
TF perturbations	heart	baseline	0.0325910779691319	0.5	
TF perturbations	heart	dummy	0.035439892613548706	0.5129310675696458	
regNetwork	heart	baseline	0.010416915057491547	0.5	
regNetwork	heart	dummy	0.011394647878809746	0.5148571158434503	


<IPython.core.display.Javascript object>